# Setup

## Libraries

### Audio

In [5]:
!pip install pretty-midi
!pip install essentia
!pip install resampy
!pip install librosa

In [6]:
!sudo apt install -y fluidsynth
!pip install midi2audio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.1.7-1.1).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.
  Using cached midi2audio-0.1.1-py2.py3-none-any.whl.metadata (5.7 kB)
Using cached midi2audio-0.1.1-py2.py3-none-any.whl (8.7 kB)


### Modelling

In [7]:
pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /var/tmp/pip-req-build-o9qjvay3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /var/tmp/pip-req-build-o9qjvay3
  Resolved https://github.com/huggingface/transformers.git to commit 4f27ee936a861f56f32ea6db138978b274008006
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 7.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 90.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 35.4 MB/s eta 0:00:00


## Imports

In [9]:
# Audio
import os
import librosa
import glob
import pretty_midi
from midi2audio import FluidSynth

# Model
from transformers import Pop2PianoForConditionalGeneration, Pop2PianoProcessor

# Data

In [18]:
os.getcwd()
!ls

'Data filtering.ipynb'
'Model exploration - pop2piano and T5.ipynb'
'Removing midi tracks.ipynb'
 Sequence_of_functions_copy-240308.ipynb


In [20]:
data_folder = "../data"
midi_file_paths = glob.glob(f"{data_folder}/.mid")[:10]
midi_files = [pretty_midi.PrettyMIDI(p) for p in midi_file_paths]
print(f"Working with {len(midi_files)} files.")

Working with 0 files.


In [31]:
test_title = "Hotel_California_1"
single_midi = pretty_midi.PrettyMIDI(f"{data_folder}/{test_title}.mid")

In [32]:
fs = FluidSynth()
fs.midi_to_audio(f"{data_folder}/{test_title}.mid", f'../test_data/{test_title}.wav')

Parameter '/home/jupyter/.fluidsynth/default_sound_font.sf2' not a SoundFont or MIDI file or error occurred identifying it.


FluidSynth runtime version 2.1.7
Copyright (C) 2000-2021 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file '../test_data/Hotel_California_1.wav'..


# Pop2Piano

## Out of the box prediction

In [35]:
audio, sr = librosa.load(f"../test_data/{test_title}.wav", sr=44100)  # feel free to change the sr to a suitable value.
model = Pop2PianoForConditionalGeneration.from_pretrained("sweetcocoa/pop2piano")
processor = Pop2PianoProcessor.from_pretrained("sweetcocoa/pop2piano")

inputs = processor(audio=audio, sampling_rate=sr, return_tensors="pt")
model_output = model.generate(input_features=inputs["input_features"], composer="composer1")
tokenizer_output = processor.batch_decode(
    token_ids=model_output, feature_extractor_output=inputs
)["pretty_midi_objects"][0]
tokenizer_output.write(f"../test_data/{test_title}_pop2piano.mid")

### Make piano arrangement sound like drum



In [36]:
piano_to_drum_hack = pretty_midi.PrettyMIDI(f"../test_data/{test_title}_pop2piano.mid")

In [44]:
piano_to_drum_hack.instruments[0].is_drum =True
piano_to_drum_hack.instruments

[Instrument(program=0, is_drum=True, name="")]

In [45]:
piano_to_drum_hack.write(f"../test_data/{test_title}_pop2piano_drum_hack.mid")